In [2]:
import pandas as pd
import numpy as np
import openpyxl

In [10]:
df = pd.read_excel('../dataset/DTX_data.xlsx', sheet_name=[ 'Data(수정)'])

In [11]:
df

{'Data(수정)':      1: 성공(금연시도 경험 있으며 최장 금연기간 6개월 이상)\n\n0: 실패(금연시도 경험 없음 or 금연시도 경험 있으나 최장 금연기간 6개월 미만)  \
 0                                                금연성공여부                                     
 1                                                     0                                     
 2                                                     0                                     
 3                                                     1                                     
 4                                                     1                                     
 ...                                                 ...                                     
 1051                                                  0                                     
 1052                                                  0                                     
 1053                                                  0                                     
 1054                                           